## **Pemantauan Muatan Suspensi Air (TSS) Menggunakan Citra Sentinel-2**

In [ ]:
import ee

In [ ]:
ee.Authenticate()

In [ ]:
# Ganti 'ee-yourproject' dengan nama project Anda
ee.Initialize(project='ee-yourproject')

In [ ]:
# Sesuaikan dengan tanggal pengamatan Anda

tgl_awal = '2019-06-25'
tgl_akhir = '2019-07-05'
persen_awan_maksimum = 50

s2_col = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterDate(tgl_awal,tgl_akhir)
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE',persen_awan_maksimum))
)

In [ ]:
# Mengembalikan nilai pixel Sentinel-2 ke nilai reflektan absolut
s2_image = s2_col.median().divide(10000)

In [ ]:
# Menentukan titik yang menjadi fokus observasi
# Sesuaikan dengan lokasi Anda, koordinat bisa Anda ambil dari Google Map
titik_observasi = ee.Geometry.Point([114.490637, -3.520689])

In [ ]:
import geemap

rgb_vis = {'min': 0, 'max': 0.1, 'bands':['B4','B3','B2']}

image_map = geemap.Map()
image_map.centerObject(titik_observasi, 11)
image_map.add_layer(s2_image, rgb_vis, 'Sentinel-2 RGB')
image_map

In [ ]:
# Kalkulasi Modified Normalized Difference Water Index (MNDWI)

mndwi = s2_image.normalizedDifference(['B3','B11']).rename('MNDWI')

In [ ]:
# Visualisasi MNDWI

mndwi_vis = {'min': -1, 'max': 1, 'palette': 'gist_earth_r'}

mndwi_map = geemap.Map()
mndwi_map.centerObject(titik_observasi, 12)
mndwi_map.add_layer(mndwi, mndwi_vis, 'MNDWI')
mndwi_map

In [ ]:
# Normalized Difference Turbidity Index (NDTI)

ndti = s2_image.normalizedDifference(['B4','B3']).rename('NDTI')

In [ ]:
# Thresholding MNDWI untuk mengekstrak tubuh air
water = mndwi.gt(0)

# Masking kekeruhan air (turbidity)
turbidity = ndti.multiply(water)
turbidity = turbidity.updateMask(turbidity.neq(0)).rename('Turbidity')

In [ ]:
# Membuat garis jalur pemantauan muatan suspensi
# Sesuaikan dengan lokasi Anda, koordinat bisa Anda ambil dari Google Map

garis_pemantau = ee.Geometry.LineString([[114.437644, -3.521386], [114.505488, -3.556828]], proj='EPSG:4326')

In [ ]:
# Visualisasi kekeruhan air

turbidity_vis = {'min': 0, 'max': 0.2, 'palette': 'jet'}

turbidity_map = geemap.Map()
turbidity_map.centerObject(titik_observasi, 11)
turbidity_map.add_layer(turbidity, turbidity_vis, 'Turbidity')
turbidity_map.add_layer(garis_pemantau, {}, 'Jalur Pemantauan TSS')
turbidity_map

In [ ]:
# Mengambil band 7 (Red Edge 3) Sentinel-2 MSI
re3 = s2_image.select('B7')

In [ ]:
# Menghitung muatan suspensi air (TSS)
# Berdasarkan formula Liu et al. (2017)
# Link publikasi: https://www.mdpi.com/2072-4292/9/7/761
tss = (re3.pow(1.357)).multiply(2950).rename('TSS')

# Masking TSS menggunakan tubuh air
tss = tss.multiply(water)
tss = tss.updateMask(tss.neq(0))

In [ ]:
# Mengekstrak data TSS sepanjang garis pemantau
data_tss = geemap.extract_transect(
    tss, garis_pemantau, n_segments=100, scale=10,
    reducer='mean', to_pandas=True
)
data_tss

In [ ]:
# Visualisasi grafik data TSS sepanjang garis pemantau

import matplotlib.pyplot as plt

# Jangan lupa ganti judul grafik sesuai wilayah Anda
judul_grafik = 'Pemantauan Muatan Suspensi Muara Sungai Barito'

plt.style.use('seaborn-v0_8')

fig = plt.figure()
plt.plot(data_tss['distance'], data_tss['mean'])
plt.xlabel('Jarak (meter)')
plt.ylabel('Total Suspended Solid (TSS) (mg/Liter)')
plt.title(judul_grafik)
plt.show()